In [156]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,KFold
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder


In [157]:
df_train = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter= False)
df_test = pd.read_csv('../../../datasets/parte2/teste/dataset_prepared.csv')

In [158]:
lb_make = LabelEncoder()
df_train['injection_encoded'] = lb_make.fit_transform(df_train['injection'])
df_train.drop('injection',inplace= True,axis=1)

X = df_train.drop('injection_encoded', axis=1)
y = df_train['injection_encoded']

In [159]:
df_train.shape

(11016, 22)

In [160]:
df_test.shape

(2256, 21)

In [161]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [162]:
%pip install -upgrade keras


Usage:   
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [163]:
 # %pip install scikeras[tensorflow]

In [164]:
def build_model(activation = 'relu',learning_rate =0.01):
    model = Sequential()
    model.add(Dense(16,input_dim=21, activation = activation ))
    model.add(Dense(8,activation=activation))
    model.add(Dense(1,activation=activation)) # output 
    #Compile the model
    model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.optimizers.Adam(learning_rate),
    metrics = ['accuracy'])
    return model 

In [165]:
model = build_model()
model.summary()


Model: "sequential_71"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_213 (Dense)           (None, 16)                352       
                                                                 
 dense_214 (Dense)           (None, 8)                 136       
                                                                 
 dense_215 (Dense)           (None, 1)                 9         
                                                                 
Total params: 497
Trainable params: 497
Non-trainable params: 0
_________________________________________________________________


In [166]:
optimizer = ['Adam', 'SGD','RMSProp']
param_grid = dict(optimizer=optimizer)

In [167]:
kf = KFold(n_splits=2, shuffle=True, random_state=2023)

In [168]:
model = KerasClassifier(build_fn=build_model, epochs=1, batch_size=32, verbose=0)


In [169]:
grid_search = GridSearchCV(estimator = model, param_grid= param_grid, cv = kf, refit='True',verbose = 1 )

In [170]:
print(X)
print(y)

           temp  feels_like  temp_min  temp_max  pressure  humidity  \
0      0.336788    0.361443  0.323608  0.316800     0.825  0.765432   
1      0.324698    0.349494  0.376606  0.313310     0.850  0.777778   
2      0.311128    0.336857  0.357602  0.301595     0.825  0.802469   
3      0.303232    0.329504  0.293897  0.289133     0.825  0.814815   
4      0.303232    0.329963  0.279176  0.289133     0.825  0.827160   
...         ...         ...       ...       ...       ...       ...   
11011  0.379472    0.408088  0.433887  0.358674     0.600  0.827160   
11012  0.375771    0.404871  0.357602  0.358175     0.600  0.839506   
11013  0.323958    0.355469  0.344486  0.315553     0.625  0.913580   
11014  0.311128    0.342831  0.344486  0.301595     0.625  0.925926   
11015  0.311128    0.343520  0.344486  0.301595     0.625  0.938272   

       wind_speed  clouds_all      hour    normal  ...  self-consumption  \
0        0.172101        0.87  0.000000  0.000000  ...               0.

In [171]:
grid_search.fit(X,y)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``buil

GridSearchCV(cv=KFold(n_splits=2, random_state=2023, shuffle=True),
             estimator=KerasClassifier(batch_size=32, build_fn=<function build_model at 0x284aefac0>, verbose=0),
             param_grid={'optimizer': ['Adam', 'SGD', 'RMSProp']}, refit='True',
             verbose=1)

In [172]:
best_mlp_model = grid_search.best_estimator_
print(best_mlp_model)

KerasClassifier(
	model=None
	build_fn=<function build_model at 0x284aefac0>
	warm_start=False
	random_state=None
	optimizer=Adam
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)


In [173]:
best_mlp_model.fit(X,y,epochs = 20, validation_data = (X,y), verbose = 1)

Epoch 1/20


/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


345/345 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0979 - val_loss: nan - val_accuracy: 0.1001
Epoch 2/20
345/345 [==============================] - 0s 910us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 3/20
345/345 [==============================] - 0s 870us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 4/20
345/345 [==============================] - 0s 861us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 5/20
345/345 [==============================] - 0s 859us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 6/20
345/345 [==============================] - 0s 835us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 7/20
345/345 [==============================] - 0s 833us/step - loss: nan - accuracy: 0.1001 - val_loss: nan - val_accuracy: 0.1001
Epoch 8/20
345/345 [===========================

KerasClassifier(
	model=None
	build_fn=<function build_model at 0x284aefac0>
	warm_start=False
	random_state=None
	optimizer=Adam
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [174]:
predictions = best_mlp_model.predict(df_test)

In [175]:
df_predicoes = pd.DataFrame({'Result': predictions})
df_predicoes['RowId'] = range(1, len(predictions) + 1)
df_predicoes = df_predicoes[['RowId', 'Result']]

df_predicoes.to_csv('../../../datasets/parte2/teste/neuralNetwork.csv', index=False)